# Analyse des ventes de l'entreprise «Rester livres»

*Etude data - Projet 4 - Nalron (octobre 2019)* /
*OpenClassrooms - ENSAE-ENSAI Formation Continue*

---

### Rappel du contexte

Je suis Data Analyst d'une grande chaîne de librairie, fraîchement embauché depuis une semaine ! "Rester livres" s'est d'abord développée dans une grande ville de France, avec plusieurs magasins, jusqu'à décider d'ouvrir une boutique en ligne. Son approche de la vente de livres en ligne, basée sur des algorithmes de recommandation, lui a valu un franc succès ! Voyons plus en détail l'approche Business de l'exploitation…

In [1]:
#Import des librairies Python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import math as mth

In [48]:
#Paramètres graphiques
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [14, 7]
plt.rcParams['font.size'] = 16

In [4]:
#Import des fichiers csv extraits directement de la base de données de l’entreprise
df_transactions = pd.read_csv('p4_data/transactions.csv') #les ventes (appelées “Transactions”)
df_products = pd.read_csv('p4_data/products.csv') #la liste des produits
df_customers = pd.read_csv('p4_data/customers.csv') #la liste des clients

In [5]:
#Visualisation rapide de nos 3 dataframes
print(df_products.info())
print("----")
print(df_customers.info())
print("----")
print(df_transactions.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3287 entries, 0 to 3286
Data columns (total 3 columns):
id_prod    3287 non-null object
price      3287 non-null float64
categ      3287 non-null int64
dtypes: float64(1), int64(1), object(1)
memory usage: 77.2+ KB
None
----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8623 entries, 0 to 8622
Data columns (total 3 columns):
client_id    8623 non-null object
sex          8623 non-null object
birth        8623 non-null int64
dtypes: int64(1), object(2)
memory usage: 202.2+ KB
None
----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337016 entries, 0 to 337015
Data columns (total 4 columns):
id_prod       337016 non-null object
date          337016 non-null object
session_id    337016 non-null object
client_id     337016 non-null object
dtypes: object(4)
memory usage: 10.3+ MB
None


# Mission 1 : nettoyage des données…

## Traitement des valeurs manquantes

In [6]:
#Méthode .isnull() (ou .isna())sur les données produits, renvoi de booléens par colonne
df_products.isnull().any()

id_prod    False
price      False
categ      False
dtype: bool

*Aucune valeur manquante dans df_products.*

In [7]:
#Méthode .isnull() sur les données clients, renvoi de booléens par colonne
df_customers.isnull().any()

client_id    False
sex          False
birth        False
dtype: bool

*Aucune valeur manquante dans df_customers.*

In [8]:
#Méthode .isnull() sur les données transactions, renvoi de booléens par colonne
df_transactions.isnull().any()

id_prod       False
date          False
session_id    False
client_id     False
dtype: bool

*Aucune valeur manquante dans df_transactions.*

## Traitement des valeurs aberrantes et atypiques 
Par valeur aberrante, on s'attend à identifier des valeurs vraisemblablement fausses, et pour les valeurs atypiques elles restent possibles, pas forcément fausses.

### Analyse et correction éventuelle du dataframe *df_products* :

In [9]:
#Tri rapide sur les 5 premières lignes du dataframe df_product (méthode .sort_values())
df_products.sort_values('id_prod', ascending = False).head()

,id_prod,price,categ
731,T_0,-1.00,0
3188,2_99,84.99,2
3088,2_98,149.74,2
2698,2_97,160.99,2
2576,2_96,47.91,2


In [10]:
#Tri rapide sur les 5 dernières lignes du dataframe df_products
df_products.sort_values('id_prod', ascending = False).tail()

,id_prod,price,categ
922,0_1000,6.84,0
663,0_100,20.60,0
2691,0_10,17.95,0
803,0_1,10.99,0
1001,0_0,3.75,0


*Ligne index 731 id_prod T_0 et price ne sont pas acceptables (valeur test négative).*

In [11]:
#Vérification par restriction des valeurs négatives (ou nulles) dans df_products
df_products[df_products.price <= 0]

,id_prod,price,categ
731,T_0,-1.0,0


In [12]:
#Suppression de la ligne index 731, la valeur n'a aucune signification logique
df_products = df_products[df_products.id_prod != 'T_0']

*La liste des produits vendus recense une seule valeur négative (- 1€) très certainement liée à une transaction de test.*

In [13]:
#Vérification rapide de la cohérence des prix produits
print(df_products.price.min())
print(df_products.price.max())

0.62
300.0


*Aucune anomalie sur les prix min. et max., les prix produits en queue de distribution sont plausibles.*

### Analyse et correction éventuelle du dataframe *df_customers* :

In [14]:
#Tri rapide sur les 5 premières lignes du dataframe df_customers (méthode .sort_values())
df_customers.sort_values('client_id', ascending = False).head()

,client_id,sex,birth
8494,ct_1,m,2001
2735,ct_0,f,2001
7358,c_999,m,1964
2145,c_998,m,2001
94,c_997,f,1994


In [15]:
#Tri rapide sur les 5 dernières lignes du dataframe df_customers
df_customers.sort_values('client_id', ascending = False).tail()

,client_id,sex,birth
3426,c_1001,m,1982
8472,c_1000,f,1966
2137,c_100,m,1992
6894,c_10,m,1956
4299,c_1,m,1955


*client_id, ct_1 et ct_2 semblent ne pas avoir la même forme d'écriture, le "ct" ressemble à un identifiant de test.*

In [16]:
#Suppression des deux lignes client_id ct_1 et ct_2
df_customers = df_customers[(df_customers.client_id != 'ct_0') & (df_customers.client_id != 'ct_1')]

In [17]:
#Vérification rapide de la cohérence des âges clients
print(df_customers.sort_values(by='birth', ascending=False).head())
print(df_customers.sort_values(by='birth', ascending=True).head())

     client_id sex  birth
7078    c_8245   f   2004
6330    c_2854   m   2004
445     c_2627   m   2004
1787    c_7916   f   2004
2947     c_308   f   2004
     client_id sex  birth
4569     c_577   m   1929
2491    c_8362   f   1929
418     c_5302   m   1929
1086    c_3218   f   1930
5138    c_7232   f   1930


*Les plus jeunes clients ont 18 ans, les plus âgés 93 ans. Aucune anomalie.*

### Analyse et correction éventuelle du dataframe *df_transactions* :

In [18]:
#Tri rapide sur les 5 premières lignes du dataframe df_transactions (méthode .sort_values())
df_transactions.sort_values('client_id', ascending = False).head()

,id_prod,date,session_id,client_id
298399,T_0,test_2021-03-01 02:30:02.237423,s_0,ct_1
57261,T_0,test_2021-03-01 02:30:02.237439,s_0,ct_1
77758,T_0,test_2021-03-01 02:30:02.237429,s_0,ct_1
2895,T_0,test_2021-03-01 02:30:02.237414,s_0,ct_1
250656,T_0,test_2021-03-01 02:30:02.237431,s_0,ct_1


In [19]:
#Tri rapide sur les 5 dernières lignes du dataframe df_transactions
df_transactions.sort_values('client_id', ascending = False).tail()

,id_prod,date,session_id,client_id
83342,0_1429,2021-10-15 11:28:24.523566,s_105105,c_1
321196,0_1547,2021-09-08 08:27:49.586711,s_86739,c_1
298829,0_1090,2021-12-19 02:44:12.827475,s_136532,c_1
28406,1_713,2021-11-15 20:40:00.586010,s_120172,c_1
126857,0_1571,2022-02-01 01:47:04.355850,s_158128,c_1


*Identification de nouvelles valeurs "test".* 

*Valeurs directement liées aux client_id ct_1 et ct_2 identifiées précédemment.*


In [20]:
#Suppression des valeurs test
df_transactions = df_transactions[df_transactions.id_prod != 'T_0']

In [21]:
#Conversion des valeurs de la colonne 'date' dans le bon format date (méthode .to_datetime())
df_transactions['date'] = pd.to_datetime(df_transactions.date, format='%Y-%m-%d %H:%M:%S', errors = 'coerce')

#Vérification du type de données
df_transactions.dtypes

id_prod               object
date          datetime64[ns]
session_id            object
client_id             object
dtype: object

In [22]:
#L'argument "errors = 'coerce'" de la méthode renvoie (NaT) si la conversion n'est pas possible
#Vérification des éventuelles valeurs manquantes
df_transactions.date.isnull().sum()

0

## Traitement des éventuels doublons

In [23]:
#Méthode .duplicated() pour identifier la présence de doublons
print(df_products.duplicated().sum())
print(df_customers.duplicated().sum())
print(df_transactions.duplicated().sum())

0
0
0


*Aucun doublon détecté dans nos 3 dataframes.*

En conclusion, à ce stade de l'analyse, seules des valeurs dites "test" ont été supprimées pour ne pas fausser l'étude. Il n'y a pas vraiment de valeurs atypiques, les prix en queue de distribution restent cohérents, ainsi que les clients les plus âgés.

[Voir la suite du projet : Analyse univariée / bivariée, indicateurs de tendance centrale et de dispersion;
analyse de concentration via une courbe de Lorenz et un indice de Gini, etc…](https://github.com/nalron/project_business_sales_analysis/blob/french_version/p4_notebook02.ipynb) 